In [330]:
#此段下載資料，如果沒有要下載勿重複使用
"""
def real_estate_crawler(year, season):
  if year > 1000:
    year -= 1911

  # download real estate zip file
  res = requests.get("https://plvr.land.moi.gov.tw//DownloadSeason?season="+str(year)+"S"+str(season)+"&type=zip&fileName=lvr_landcsv.zip")

  # save content to file
  fname = str(year)+str(season)+'.zip'
  open(fname, 'wb').write(res.content)

  # make additional folder for files to extract
  folder = 'real_estate' + str(year) + str(season)
  if not os.path.isdir(folder):
    os.mkdir(folder)

  # extract files to the folder
  with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall(folder)

  time.sleep(10)

real_estate_crawler(101, 3)
real_estate_crawler(101, 4)

for year in range(102, 108):
  for season in range(1,5):
    print(year, season)
    real_estate_crawler(year, season)

real_estate_crawler(108, 1)
real_estate_crawler(108, 2)
"""

'\ndef real_estate_crawler(year, season):\n  if year > 1000:\n    year -= 1911\n\n  # download real estate zip file\n  res = requests.get("https://plvr.land.moi.gov.tw//DownloadSeason?season="+str(year)+"S"+str(season)+"&type=zip&fileName=lvr_landcsv.zip")\n\n  # save content to file\n  fname = str(year)+str(season)+\'.zip\'\n  open(fname, \'wb\').write(res.content)\n\n  # make additional folder for files to extract\n  folder = \'real_estate\' + str(year) + str(season)\n  if not os.path.isdir(folder):\n    os.mkdir(folder)\n\n  # extract files to the folder\n  with zipfile.ZipFile(fname, \'r\') as zip_ref:\n    zip_ref.extractall(folder)\n\n  time.sleep(10)\n\nreal_estate_crawler(101, 3)\nreal_estate_crawler(101, 4)\n\nfor year in range(102, 108):\n  for season in range(1,5):\n    print(year, season)\n    real_estate_crawler(year, season)\n\nreal_estate_crawler(108, 1)\nreal_estate_crawler(108, 2)\n'

In [331]:
#爬取了實價登入所有歷史數據
import requests
import os
import zipfile
import time
from datetime import datetime

In [332]:
"""'公寓(5樓含以下無電梯)', '透天厝', '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)',
       '華廈(10層含以下有電梯)',"""

"'公寓(5樓含以下無電梯)', '透天厝', '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)',\n       '華廈(10層含以下有電梯)',"

In [333]:
"""x_lvr_land_a：房屋買賣交易
x_lvr_land_b：新成屋交易
x_lvr_land_c：租房交易
"""

'x_lvr_land_a：房屋買賣交易\nx_lvr_land_b：新成屋交易\nx_lvr_land_c：租房交易\n'

In [334]:
#本案例只對二手房市做預測
import os
import pandas as pd
import numpy as np
import math
dirs = [d for d in os.listdir() if d[:4] == 'real' and int(d[11:14]) >=105 ]

dfs = []

for d in dirs:
    print(d)

    df = pd.read_csv(os.path.join(d,'a_lvr_land_a.csv'), index_col=False)
    df['Q'] = d[-1]
    dfs.append(df.iloc[1:])
    
df = pd.concat(dfs, sort=True)

real_estate1051
real_estate1052
real_estate1053
real_estate1054
real_estate1061
real_estate1062
real_estate1063
real_estate1064
real_estate1071
real_estate1072
real_estate1073
real_estate1074
real_estate1081
real_estate1082


In [335]:
df = df.reset_index(drop=True)

In [336]:
#資訊還必須再經過處理，才會讓我們待會的資料分析更好下手

In [337]:
def process(df):
  df['year'] = df['交易年月日'].str[:-4].astype(int) + 1911
  df['month'] = df['交易年月日'].str[-4:-2].astype(int)
  df['day'] = df['交易年月日'].str[-2:].astype(int)
  # 不同名稱同項目資料合併
  df['單價元/平方公尺'].fillna(df['單價元/平方公尺'], inplace=True)
  df.drop(columns='單價元/平方公尺')
  # 平方公尺換成坪
  df['單價元/平方公尺'] = df['單價元/平方公尺'].astype(float)
  df['建物型態2'] = df['建物型態'].str.split('(').str[0]
  # 刪除有備註之交易（多為親友交易、價格不正常之交易）
  df = df[df['備註'].isnull()]

  df.index = pd.to_datetime((df['交易年月日'].str[:-4].astype(int) + 1911).astype(str) + df['交易年月日'].str[-4:] ,errors='coerce')


process(df)

In [338]:
"""
set:集合
集合（set）是一个无序的不重复元素序列。

可以使用大括号 { } 或者 set() 函数创建集合，
注意：创建一个空集合必须用 set() 而不是 { }，因为 { } 是用来创建一个空字典。
"""

'\nset:集合\n集合（set）是一个无序的不重复元素序列。\n\n可以使用大括号 { } 或者 set() 函数创建集合，\n注意：创建一个空集合必须用 set() 而不是 { }，因为 { } 是用来创建一个空字典。\n'

In [339]:
#篩選建物型態
df=df[df["建物型態"].isin(['公寓(5樓含以下無電梯)',  '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)','華廈(10層含以下有電梯)'])]

In [340]:
#check
df["建物型態"].unique()

array(['公寓(5樓含以下無電梯)', '套房(1房1廳1衛)', '住宅大樓(11層含以上有電梯)', '華廈(10層含以下有電梯)'],
      dtype=object)

In [341]:
set(df['鄉鎮市區'])

{'中山區',
 '中正區',
 '信義區',
 '內湖區',
 '北投區',
 '南港區',
 '士林區',
 '大同區',
 '大安區',
 '文山區',
 '松山區',
 '萬華區'}

In [342]:
df.columns

Index(['Q', '主要建材', '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '單價元/平方公尺',
       '單價元平方公尺', '土地區段位置/建物區段門牌', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '建物型態',
       '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛', '建物現況格局-隔間', '建物移轉總面積平方公尺',
       '建築完成年月', '有無管理組織', '移轉層次', '編號', '總價元', '總樓層數', '車位移轉總面積平方公尺', '車位總價元',
       '車位類別', '都市土地使用分區', '鄉鎮市區', '非都市土地使用分區', '非都市土地使用編定', 'year', 'month',
       'day', '建物型態2'],
      dtype='object')

In [343]:
df.主要建材.unique()

array(['鋼筋混凝土造', '加強磚造', '見其他登記事項', '鋼骨鋼筋混凝土造', '磚造', '見使用執照', nan,
       '鋼骨混凝土造', '預力混凝土造', '壁式預鑄鋼筋混凝土造', '鋼筋混凝土加強磚造', '土磚石混合造', '木造',
       '鋼造'], dtype=object)

In [344]:
try:
    special_material = ['見其他登記事項','見使用執照', nan]
except:
    special_material = []

In [345]:
#可以發現nan無法直接進入list，所以isin沒辦法處理nan
special_material

[]

In [346]:
def deal_special_dump(df,col,specialcategory):
    specialcategory.append('NaN')
    df[col].replace(pd.np.nan,'NaN',inplace=True)
    df=df[~df[col].isin(specialcategory)]
    print("unique col now:",df[col].unique())
    return df

def deal_special_dump_nan0(df,col,specialcategory=[]):
    if df[col].iloc[0].isdigit() is True:
        df[col].replace(pd.np.nan,0,inplace=True)
    else:
        df[col].replace(pd.np.nan,'NaN',inplace=True)
    df=df[~df[col].isin(specialcategory)]
    print("unique col now:",df[col].unique())
    return df

def deal_positive_cat(df,col,specialcategory):
    df=df[df[col].isin(specialcategory)]
    print("unique col now:",df[col].unique())
    return df

def deal_positive_contain(df,col,specialcategory):
    df[col].replace(pd.np.nan,'NaN',inplace=True)
    df=df[df[col].str.contains('|'.join(specialcategory))]
    return df

def deal_positive_contain(df,col,specialcategory):
    #df[col].replace(pd.np.nan,'NaN',inplace=True)
    df=df[df[col].str.contains('|'.join(specialcategory))]
    return df
def deal_negative_contain(df,col,specialcategory):
    df[col].replace(pd.np.nan,'NaN',inplace=True)
    specialcategory.append('NaN')
    df=df[~df[col].str.contains('|'.join(specialcategory))]
    return df

In [347]:
df.主要建材.unique()

array(['鋼筋混凝土造', '加強磚造', '見其他登記事項', '鋼骨鋼筋混凝土造', '磚造', '見使用執照', nan,
       '鋼骨混凝土造', '預力混凝土造', '壁式預鑄鋼筋混凝土造', '鋼筋混凝土加強磚造', '土磚石混合造', '木造',
       '鋼造'], dtype=object)

In [348]:
df=deal_special_dump(df,"主要建材",['見使用執照', '見其他登記事項'])

unique col now: ['鋼筋混凝土造' '加強磚造' '鋼骨鋼筋混凝土造' '磚造' '鋼骨混凝土造' '預力混凝土造' '壁式預鑄鋼筋混凝土造'
 '鋼筋混凝土加強磚造' '土磚石混合造' '木造' '鋼造']


In [349]:
df.主要用途.unique()

array(['住家用', '商業用', '見使用執照', '見其他登記事項', nan, '住商用', '工業用', '住工用', '停車空間',
       '工商用', '國民住宅'], dtype=object)

In [350]:
df['主要用途'].iloc[0]

'住家用'

In [351]:
df=deal_special_dump(df,"主要用途",['見使用執照', '見其他登記事項'])

unique col now: ['住家用' '商業用' '住商用' '工業用' '住工用' '停車空間' '工商用' '國民住宅']


In [352]:
df.交易標的.unique()

array(['房地(土地+建物)', '房地(土地+建物)+車位', '建物'], dtype=object)

In [353]:
df.交易年月日

0        1041005
32       1041003
33       1040921
34       1041007
35       1040925
36       1041012
37       1040902
39       1041020
40       1040916
41       1041003
42       1040927
43       1040823
44       1041006
45       1040918
47       1041020
48       1041018
49       1040827
52       1041019
53       1041016
54       1041012
55       1041008
60       1040922
61       1041006
62       1000224
63       1040912
64       1041020
66       1041014
67       1041008
68       1020430
69       1041013
          ...   
74656    1080501
74658    1080406
74659    1080401
74660    1080225
74661    1071207
74662    1080401
74663    1080323
74664    1080405
74665    1071029
74666    1080312
74669    1080412
74672    1080408
74673    1080326
74674    1080413
74675    1080331
74676    1080418
74678    1080408
74679    1080227
74683    1080429
74684    1080422
74685    1080328
74686    1080328
74687    1080115
74688    1080410
74690    1060422
74691    1080410
74693    1080330
74696    10802

In [354]:
df.year

0        2015
32       2015
33       2015
34       2015
35       2015
36       2015
37       2015
39       2015
40       2015
41       2015
42       2015
43       2015
44       2015
45       2015
47       2015
48       2015
49       2015
52       2015
53       2015
54       2015
55       2015
60       2015
61       2015
62       2011
63       2015
64       2015
66       2015
67       2015
68       2013
69       2015
         ... 
74656    2019
74658    2019
74659    2019
74660    2019
74661    2018
74662    2019
74663    2019
74664    2019
74665    2018
74666    2019
74669    2019
74672    2019
74673    2019
74674    2019
74675    2019
74676    2019
74678    2019
74679    2019
74683    2019
74684    2019
74685    2019
74686    2019
74687    2019
74688    2019
74690    2017
74691    2019
74693    2019
74696    2019
74697    2019
74698    2019
Name: year, Length: 46636, dtype: int32

In [355]:
df = deal_positive_cat(df,"year",[2015,2016,2017,2018,2019])


unique col now: [2015 2016 2017 2018 2019]


In [356]:
#要修改
df.交易標的.unique()

array(['房地(土地+建物)', '房地(土地+建物)+車位', '建物'], dtype=object)

In [357]:
df.交易筆棟數.unique()

array(['土地1建物1車位0', '土地2建物1車位0', '土地1建物1車位1', '土地5建物1車位0', '土地1建物1車位2',
       '土地6建物1車位0', '土地3建物1車位1', '土地1建物2車位1', '土地2建物1車位1', '土地3建物1車位0',
       '土地5建物2車位1', '土地2建物1車位2', '土地4建物1車位0', '土地1建物1車位4', '土地1建物2車位0',
       '土地7建物1車位0', '土地0建物1車位1', '土地2建物2車位1', '土地0建物1車位0', '土地1建物2車位2',
       '土地8建物1車位0', '土地3建物3車位0', '土地1建物1車位3', '土地2建物2車位2', '土地2建物2車位0',
       '土地11建物1車位0', '土地2建物1車位4', '土地0建物1車位3', '土地3建物1車位2', '土地1建物3車位0',
       '土地2建物3車位1', '土地4建物1車位1', '土地1建物6車位4', '土地4建物1車位2', '土地0建物2車位0',
       '土地5建物1車位1', '土地2建物1車位3', '土地1建物1車位6', '土地1建物11車位0', '土地1建物5車位0',
       '土地1建物4車位0', '土地1建物12車位0', '土地4建物2車位0', '土地0建物1車位4', '土地10建物1車位0',
       '土地16建物1車位1', '土地3建物2車位0', '土地2建物42車位0', '土地2建物12車位0', '土地2建物1車位5',
       '土地2建物3車位0', '土地0建物1車位2', '土地3建物1車位3', '土地2建物3車位2', '土地1建物1車位5',
       '土地13建物1車位0', '土地1建物2車位4', '土地2建物4車位0', '土地4建物2車位1', '土地0建物2車位2',
       '土地1建物3車位3', '土地2建物2車位3', '土地0建物3車位0', '土地1建物2車位6', '土地1建物2車位5',
       '土地2建物5車位3', '土地1建物3車位4', '土地12建物1車位0', '土地0建物4車位

In [358]:
df = df[df['備註'].isnull()]

In [359]:
df['土地區段位置/建物區段門牌'][100:200]

355         臺北市內湖區陽光街161巷1弄1~30號
357          臺北市內湖區成功路二段361~390號
360         臺北市中山區南京東路一段13巷1~30號
361           臺北市中山區建國北路一段31~60號
365               臺北市中正區泰安街1~30號
369            臺北市大同區重慶北路一段1~30號
372       臺北市文山區興隆路四段58巷26弄1~30號
374            臺北市文山區興德路20巷1~30號
387            臺北市中正區?門街121~150號
389         臺北市文山區木新路三段95巷31~60號
391        臺北市萬華區和平西路三段298巷1~30號
402    臺北市內湖區民權東路六段280巷69弄61~90號
403        臺北市中山區新生北路二段60巷31~60號
409         臺北市中山區中山北路二段16巷1~30號
410     臺北市內湖區民權東路六段180巷121~150號
412               臺北市內湖區金湖路1~30號
414           臺北市南港區福德街310巷1~30號
415         臺北市信義區和平東路三段211~240號
418            臺北市松山區延吉街46巷1~30號
419             臺北市信義區松隆路9巷1~30號
425        臺北市士林區德行東路74巷15弄1~30號
426              臺北市士林區菁山路61~90號
427              臺北市士林區菁山路61~90號
428          臺北市北投區大度路三段271~300號
429          臺北市北投區石牌路一段58巷1~30號
432        臺北市士林區延平北路六段392巷1~30號
434          臺北市北投區東華街一段361~390號
435          臺北市北投區東華街一段361~390號
436          臺北市北投區東華街一段361~390號
437              臺北市北投區中山路31~60號
          

In [360]:
df.土地移轉總面積平方公尺.unique()

array(['25.44', '39.2', '12.4', ..., '62.19', '16.68', '99.47'],
      dtype=object)

In [411]:
df.土地移轉總面積平方公尺 = df.土地移轉總面積平方公尺.astype(float)

In [361]:
df.建物型態.unique()

array(['公寓(5樓含以下無電梯)', '華廈(10層含以下有電梯)', '住宅大樓(11層含以上有電梯)', '套房(1房1廳1衛)'],
      dtype=object)

In [362]:
df['建物現況格局-廳'].unique()

array(['2', '1', '0', '3', '6', '4', '9', '80', '24', '5', '7', '8', '10'],
      dtype=object)

In [412]:
df['建物現況格局-廳']=df['建物現況格局-廳'].astype(int)

In [363]:
df['建物現況格局-房'].unique()

array(['3', '2', '4', '0', '1', '5', '6', '7', '9', '140', '39', '8',
       '10', '11', '12', '32', '20'], dtype=object)

In [ ]:
df['建物現況格局-房']=df['建物現況格局-房'].astype(int)

In [364]:
df['建物現況格局-衛'].unique()

array(['2', '1', '0', '3', '5', '6', '4', '7', '8', '75', '23', '10', '9',
       '30', '22'], dtype=object)

In [413]:
df['建物現況格局-衛']=df['建物現況格局-衛'].astype(int)

In [414]:
df.建物移轉總面積平方公尺[0:10]

0     107.40
35    148.95
39     46.08
43    326.63
44     90.91
45    102.76
48     99.08
53    151.28
54    104.74
64    102.16
Name: 建物移轉總面積平方公尺, dtype: float64

In [415]:
df.建築完成年月[0:10]

0     0701231
35    0830530
39    0000001
43    0981126
44    0710622
45    0690515
48    0690422
53    0690324
54    0691024
64    0690428
Name: 建築完成年月, dtype: object

adjust
""" 	建築完成年月 	
4150 	0005807 	
12013 	0005305 	
15153 	8012500 	
"""

In [416]:
df['建築完成年月'][15153]='0800125'

D:\Users\Bojun_Zhang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [417]:
df.columns

Index(['Q', '主要建材', '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '單價元/平方公尺',
       '單價元平方公尺', '土地區段位置/建物區段門牌', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '建物型態',
       '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛', '建物現況格局-隔間', '建物移轉總面積平方公尺',
       '建築完成年月', '有無管理組織', '移轉層次', '編號', '總價元', '總樓層數', '車位移轉總面積平方公尺', '車位總價元',
       '車位類別', '都市土地使用分區', '鄉鎮市區', '非都市土地使用分區', '非都市土地使用編定', 'year', 'month',
       'day', '建物型態2', 'b_year', 'b_month', 'b_day', 'b_time', 'time', '屋齡',
       '總樓層數2', '移轉層次_2', '移轉層次_3', '扣除車位建物移轉總面積平方公尺'],
      dtype='object')

In [418]:

def fill_back_0(strin):
    strin=int(strin)
    #if type(strin)!=str:
    if strin ==0: strin=1
    if int(math.log10(strin))+1==3:
        strin=strin*10000
    elif int(math.log10(strin))+1==4:
        strin=strin*100    
    elif int(math.log10(strin))+1==5:
        strin=strin*100
    strin=str(strin).zfill(7)
    return strin


In [419]:
df['建築完成年月'].str[0:4]

0        0701
35       0830
39       0000
43       0981
44       0710
45       0690
48       0690
53       0690
54       0691
64       0690
69       0000
70       0751
84       1020
85       0860
87       0740
90       0000
96       0671
101      0700
108      0700
111      1011
112      0841
113      0751
116      0950
117      0970
120      0840
122      0931
123      0590
124      0000
127      0701
128      0960
         ... 
74561    1021
74562    0670
74572    0740
74575    0681
74576    0570
74586    1061
74588    0860
74592    1011
74594    0630
74596    0990
74603    0690
74604    0000
74606    0710
74607    0890
74611    0970
74626    0970
74629    0590
74643    0690
74645    0670
74651    0780
74653    0611
74656    0610
74663    0730
74665    0590
74669    0730
74674    0830
74678    0850
74683    0900
74691    1000
74693    0991
Name: 建築完成年月, Length: 20174, dtype: object

In [420]:
df = deal_special_dump_nan0(df,'建築完成年月')
df['建築完成年月'] = df.建築完成年月.apply(fill_back_0)

df['b_year'] = (df['建築完成年月'].str[:3].fillna(0).astype(int) + 1911).astype(str).str.zfill(4)
#df['b_month'] = df['建築完成年月'].str[3:5].fillna(1).astype(int)
df['b_month'] = df['建築完成年月'].str[3:5].fillna(1).astype(str).str.zfill(2)
#df['b_day'] = df['建築完成年月'].str[5:].fillna(1).astype(int)
df['b_month'].replace([0,"00","0"],"01", inplace=True)
df['b_day'] = df['建築完成年月'].str[5:].fillna(1).astype(str).str.zfill(2)
df['b_day'].replace([0,"00","0"],"01", inplace=True)
df['b_time']=pd.to_datetime(dict(year=df.b_year, month=df.b_month, day=df.b_day))
df['b_time'] =pd.to_datetime(df.b_time,format='%m-%d-%Y')

unique col now: ['0701231' '0830530' '0000001' ... '0690321' '0590217' '0900629']


In [421]:
df[df.index.duplicated()]

,Q,主要建材,主要用途,交易年月日,交易標的,交易筆棟數,備註,單價元/平方公尺,單價元平方公尺,土地區段位置/建物區段門牌,...,b_year,b_month,b_day,b_time,time,屋齡,總樓層數2,移轉層次_2,移轉層次_3,扣除車位建物移轉總面積平方公尺


In [422]:
df[['建築完成年月','b_year','b_month','b_day']]

,建築完成年月,b_year,b_month,b_day
0,0701231,1981,12,31
35,0830530,1994,05,30
39,0000001,1911,01,01
43,0981126,2009,11,26
44,0710622,1982,06,22
45,0690515,1980,05,15
48,0690422,1980,04,22
53,0690324,1980,03,24
54,0691024,1980,10,24
64,0690428,1980,04,28


In [423]:
#df['b_time'] =  df.b_month+"-"+df.b_day+"-"+df.b_year

In [424]:
#df['b_time'][0:50]

In [425]:
df = deal_special_dump_nan0(df,'交易年月日')
df['交易年月日'] = df.交易年月日.apply(fill_back_0)

df['year'] = (df['交易年月日'].str[:3].fillna(0).astype(int) + 1911).astype(str).str.zfill(4)

#check unnormal
"""
df['month'] = df['交易年月日'].str[3:5].fillna(1).astype(int)
df['day'] = df['交易年月日'].str[5:].fillna(1).astype(int)
df[df['month']>12]
df[df['day']>31]
"""
#if no unnormal, run next


df['month'] = df['交易年月日'].str[3:5].fillna(1).astype(str).str.zfill(2)
df['month'].replace([0,"00","0"],"01", inplace=True)
df['day'] = df['交易年月日'].str[5:].fillna(1).astype(str).str.zfill(2)
df['day'].replace([0,"00","0"],"01", inplace=True)
df['time']=pd.to_datetime(dict(year=df.year, month=df.month, day=df.day))
df['time'] =pd.to_datetime(df.time,format='%m-%d-%Y')

unique col now: ['1041005' '1040925' '1041020' ... '1080419' '1080501' '1080429']


In [426]:
df['time']

0       2015-10-05
35      2015-09-25
39      2015-10-20
43      2015-08-23
44      2015-10-06
45      2015-09-18
48      2015-10-18
53      2015-10-16
54      2015-10-12
64      2015-10-20
69      2015-10-13
70      2015-10-06
84      2015-09-26
85      2015-10-14
87      2015-10-15
90      2015-10-06
96      2015-10-19
101     2015-09-01
108     2015-10-09
111     2015-08-18
112     2015-07-08
113     2015-09-03
116     2015-10-20
117     2015-09-16
120     2015-10-09
122     2015-09-21
123     2015-09-24
124     2015-10-16
127     2015-06-29
128     2015-09-01
           ...    
74561   2019-03-21
74562   2019-04-06
74572   2019-04-10
74575   2019-03-17
74576   2019-03-30
74586   2019-03-31
74588   2019-03-06
74592   2019-03-10
74594   2019-03-31
74596   2019-04-11
74603   2019-04-22
74604   2019-04-03
74606   2019-03-04
74607   2019-04-19
74611   2019-04-02
74626   2019-04-01
74629   2019-03-29
74643   2019-03-31
74645   2019-04-18
74651   2019-04-18
74653   2019-04-17
74656   2019

In [427]:
df['屋齡'] = df['time'] - df['b_time']

In [428]:
df['屋齡'] =df.屋齡/ np.timedelta64(1, 'Y')
df['屋齡']= df['屋齡'].astype(int)

In [429]:
df['屋齡'] = df['屋齡'].apply(lambda x:50 if x>50 else x)

In [430]:
df['屋齡']

0        33
35       21
39       50
43        5
44       33
45       35
48       35
53       35
54       34
64       35
69       50
70       28
84        2
85       18
87       30
90       50
96       36
101      34
108      34
111       2
112      19
113      28
116       9
117       7
120      20
122      10
123      45
124      50
127      33
128       8
         ..
74561     5
74562    40
74572    33
74575    39
74576    50
74586     1
74588    21
74592     6
74594    44
74596     8
74603    39
74604    50
74606    37
74607    18
74611    10
74626    11
74629    49
74643    38
74645    40
74651    29
74653    46
74656    46
74663    34
74665    48
74669    34
74674    24
74678    22
74683    17
74691     7
74693     8
Name: 屋齡, Length: 20174, dtype: int64

In [431]:
df['有無管理組織'].unique()

array(['無', '有'], dtype=object)

In [432]:
df["總樓層數2"]=df["總樓層數"].astype(str)

In [433]:
#先轉總樓層數!

In [434]:
#-*- coding: cp936 -*-
import re
import string

digit = {'ㄧ': 1,'一': 1 ,'壹': 1, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9}

def _chn_to_numeric(s):
    num = 0
    if s:
        if s != s:
            s="null"
        idx_q, idx_b, idx_s, idx_f = s.find('千'), s.find('百'), s.find('十'), s.find('層')
        if idx_q != -1:
            num += digit[s[idx_q - 1:idx_q]] * 1000
        if idx_b != -1:
            num += digit[s[idx_b - 1:idx_b]] * 100
        if idx_s != -1:
            # 十前忽略一的处理
            #get 如果第一個沒有值就回傳第2個
            num += digit.get(s[idx_s - 1:idx_s], 1) * 10
        if idx_f != -1:
            # 十前忽略一的处理
            if s[idx_f - 1:idx_f]!="十":
                num += digit.get(s[idx_f - 1:idx_f], 1)
        if num == 0:
            num = 1
        #if s[-1] in digit:
        #    num += digit[s[-1]]
    return num

def trans(chn):
    chn=str(chn)
    chn = chn.replace('零', '')
    idx_y, idx_w = chn.rfind('億'), chn.rfind('萬')
    if idx_w < idx_y:
        idx_w = -1
    num_y, num_w = 100000000, 10000
    if idx_y != -1 and idx_w != -1:
        return trans(chn[:idx_y]) * num_y + _chn_to_numeric(chn[idx_y + 1:idx_w]) * num_w + _chn_to_numeric(chn[idx_w + 1:])
    elif idx_y != -1:
        return trans(chn[:idx_y]) * num_y + _chn_to_numeric(chn[idx_y + 1:])
    elif idx_w != -1:
        return _chn_to_numeric(chn[:idx_w]) * num_w + _chn_to_numeric(chn[idx_w + 1:])
    return _chn_to_numeric(chn)

In [450]:
df["總樓層數"]=df["總樓層數"].apply(trans)
df["總樓層數"]=df["總樓層數"].astype(int)

In [451]:
def foo(t_floor,all_floor):
    if t_floor=="全":
        return range(1,all_floor+1)
    else:
        return t_floor

df['移轉層次'] = df.apply(lambda x: foo(x["移轉層次"], x["總樓層數"]), axis=1)

In [452]:
df['移轉層次'].unique()

array([ 2,  6,  9,  7,  3,  4,  5,  1, 12, 10, 14, 11, 13,  8, 18, 19, 15,
       16, 17, 24, 20, 23, 21, 33, 29, 26, 25, 27, 22, 28, 31],
      dtype=int64)

In [453]:
df=deal_negative_contain(df,"移轉層次",['登記事項', '，','地下','屋頂'])

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [446]:
df['移轉層次'].unique()

array(['二層', '六層', '九層', '七層', '三層', '四層', '五層', '一層', '十二層', '十層', '十四層',
       '十一層', '十三層', '八層', '十八層', '十九層', '十五層', '十六層', '十七層', '二十四層',
       '二十層', '二十三層', '二十一層', '三十三層', '二十九層', '二十六層', '二十五層', '夾層',
       '二十七層', '二十二層', '二十八層', '三十一層'], dtype=object)

In [447]:
df["移轉層次"]=df["移轉層次"].apply(trans)

In [449]:
df["移轉層次"] = df["移轉層次"].astype(int)

In [454]:
df.車位類別.unique

<bound method Series.unique of 0         NaN
35       升降平面
39        NaN
43       坡道平面
44        NaN
45        NaN
48        NaN
53        NaN
54        NaN
64        NaN
69        NaN
70       坡道平面
84       坡道平面
85        NaN
87        NaN
90        NaN
96        NaN
101       NaN
108      坡道平面
111      坡道平面
112      升降平面
113       NaN
116       NaN
117      升降機械
120       NaN
122      坡道機械
123       NaN
124       NaN
127       NaN
128      坡道平面
         ... 
74561    坡道平面
74562     NaN
74572     NaN
74575     NaN
74576     NaN
74586     NaN
74588     NaN
74592    坡道機械
74594     NaN
74596    坡道平面
74603     NaN
74604     NaN
74606     NaN
74607     NaN
74611    升降機械
74626     NaN
74629     NaN
74643     NaN
74645     NaN
74651     NaN
74653     NaN
74656     NaN
74663     NaN
74665     NaN
74669    坡道平面
74674    坡道平面
74678     NaN
74683     NaN
74691    升降平面
74693    坡道平面
Name: 車位類別, Length: 20174, dtype: object>

In [456]:
df['車位類別'].replace(pd.np.nan,'無',inplace=True)

In [458]:
df.columns

Index(['Q', '主要建材', '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '單價元/平方公尺',
       '單價元平方公尺', '土地區段位置/建物區段門牌', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '建物型態',
       '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛', '建物現況格局-隔間', '建物移轉總面積平方公尺',
       '建築完成年月', '有無管理組織', '移轉層次', '編號', '總價元', '總樓層數', '車位移轉總面積平方公尺', '車位總價元',
       '車位類別', '都市土地使用分區', '鄉鎮市區', '非都市土地使用分區', '非都市土地使用編定', 'year', 'month',
       'day', '建物型態2', 'b_year', 'b_month', 'b_day', 'b_time', 'time', '屋齡',
       '總樓層數2', '移轉層次_2', '移轉層次_3', '扣除車位建物移轉總面積平方公尺'],
      dtype='object')

In [459]:
df['建物移轉總面積平方公尺']=df['建物移轉總面積平方公尺'].astype(float)
df['車位移轉總面積平方公尺']=df['車位移轉總面積平方公尺'].astype(float)

In [460]:
df['扣除車位建物移轉總面積平方公尺'] = df['建物移轉總面積平方公尺'] - df['車位移轉總面積平方公尺']

In [461]:
df['扣除車位建物移轉總面積平方公尺']

0        107.40
35       148.95
39        46.08
43       326.63
44        90.91
45       102.76
48        99.08
53       151.28
54       104.74
64       102.16
69        29.48
70       163.25
84       230.01
85        22.53
87        86.80
90        87.45
96       105.84
101      159.80
108      144.74
111      258.01
112      163.85
113      131.48
116       46.06
117      249.69
120       76.81
122      215.98
123      104.81
124      109.33
127      116.07
128      128.53
          ...  
74561    280.94
74562     79.52
74572     87.39
74575    111.49
74576     89.68
74586     72.02
74588     60.79
74592    148.23
74594     61.84
74596    183.35
74603    120.37
74604    116.51
74606     50.74
74607    155.75
74611     58.53
74626     34.00
74629    108.74
74643     99.78
74645     63.02
74651     96.72
74653    106.14
74656     42.12
74663    130.20
74665     85.00
74669    391.62
74674     94.97
74678     31.66
74683     94.98
74691    124.50
74693    148.62
Name: 扣除車位建物移轉總面積平方公尺, L

In [462]:
df=df.drop(['編號'],axis=1)

In [463]:
df.車位移轉總面積平方公尺[0:50]

0       0.00
35      0.00
39      0.00
43      0.00
44      0.00
45      0.00
48      0.00
53      0.00
54      0.00
64      0.00
69      0.00
70     38.07
84     70.28
85      0.00
87      0.00
90      0.00
96      0.00
101     0.00
108    41.98
111    38.64
112     0.00
113     0.00
116     0.00
117    37.30
120     0.00
122    40.86
123     0.00
124     0.00
127     0.00
128     0.00
129     0.00
130    41.54
137     0.00
138     0.00
147     0.00
148     0.00
154     0.00
155     0.00
160     0.00
161     0.00
162     0.00
163     2.53
164     0.00
168     0.00
169     0.00
175    36.06
228    16.27
229    16.27
231     0.00
236     0.00
Name: 車位移轉總面積平方公尺, dtype: float64

In [464]:
df.columns

Index(['Q', '主要建材', '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '單價元/平方公尺',
       '單價元平方公尺', '土地區段位置/建物區段門牌', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '建物型態',
       '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛', '建物現況格局-隔間', '建物移轉總面積平方公尺',
       '建築完成年月', '有無管理組織', '移轉層次', '總價元', '總樓層數', '車位移轉總面積平方公尺', '車位總價元',
       '車位類別', '都市土地使用分區', '鄉鎮市區', '非都市土地使用分區', '非都市土地使用編定', 'year', 'month',
       'day', '建物型態2', 'b_year', 'b_month', 'b_day', 'b_time', 'time', '屋齡',
       '總樓層數2', '移轉層次_2', '移轉層次_3', '扣除車位建物移轉總面積平方公尺'],
      dtype='object')

In [ ]:
#圖表分析
prices = {}
for district in set(df['鄉鎮市區']):
    cond = (
        (df['主要用途'] == '住家用')
        & (df['鄉鎮市區'] == district)
        & (df['單價元平方公尺'] < df["單價元平方公尺"].quantile(0.9))
        & (df['單價元平方公尺'] > df["單價元平方公尺"].quantile(0.1))
        )
    
    groups = df[cond]['year']
    
    prices[district] = df[cond]['單價元平方公尺'].astype(float).groupby(groups).mean().loc[2012:]

prices = pd.DataFrame(prices)
prices.head()

In [ ]:
prices.plot()

In [ ]:
prices.mean(axis=1).plot()

In [ ]:
building_type_prices = {}
for building_type in set(df['建物型態2']):
    cond = (
        (df['主要用途'] == '住家用')
        & (df['單價元平方公尺'] < df["單價元平方公尺"].quantile(0.8))
        & (df['單價元平方公尺'] > df["單價元平方公尺"].quantile(0.2))
        & (df['建物型態2'] == building_type)
        )
    
    building_type_prices[building_type] = df[cond]['單價元平方公尺'].groupby(df[cond]['year']).mean().loc[2012:]
pd.DataFrame(building_type_prices)[['公寓', '住宅大樓', '套房', '華廈']].plot()

In [ ]:
for district in set(df['鄉鎮市區']):
  dfdistrict = df[df['鄉鎮市區'] == district]
  dfdistrict['單價元平方公尺'][dfdistrict['單價元平方公尺'] < 1000000].hist(bins=100, alpha=0.7)

plt.xlim(0, 1000000)

In [ ]:
#以上參考https://www.finlab.tw/